In [10]:
import os
import re
from PIL import Image
import ctypes


def parse_sos_file(sos_file_path):
    koordsys = None
    coordinates = []
    inside_coordinates = False

    with open(sos_file_path, 'r', encoding='ISO8859-10') as f:
        for line in f:
            line = line.strip()
            if line.startswith('...KOORDSYS'):
                parts = line.split()
                if len(parts) >= 2:
                    koordsys = int(parts[1])
            if line.startswith('..NØ'):
                inside_coordinates = True
                continue
            if inside_coordinates:
                if re.match(r'^\d+\s+\d+$', line):
                    x, y = line.split()
                    coordinates.append((float(x), float(y)))
                else:
                    break

    return koordsys, coordinates


def initialize_transformation(dll_path, href_file):
    if not os.path.exists(dll_path):
        raise FileNotFoundError("DLL file not found.")
    if not os.path.exists(href_file):
        raise FileNotFoundError("HREF file not found.")
    
    trans_dll = ctypes.CDLL(dll_path)
    _InitSkTrans = trans_dll._InitSkTrans
    _InitSkTrans.argtypes = [ctypes.c_char_p, ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short)]
    _InitSkTrans.restype = None

    href = href_file.encode('utf-8')
    sLen = ctypes.c_short(len(href))
    sErr = ctypes.c_short(0)
    _InitSkTrans(href, ctypes.byref(sLen), ctypes.byref(sErr))
    if sErr.value != 0:
        raise RuntimeError(f"Failed to initialize transformation. Error Code: {sErr.value}")

    return trans_dll, _InitSkTrans, trans_dll._GeoTrans


def transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2):
    x1 = ctypes.c_double(x)
    y1 = ctypes.c_double(y)
    z1 = ctypes.c_double(0.0)
    x2 = ctypes.c_double(0.0)
    y2 = ctypes.c_double(0.0)
    z2 = ctypes.c_double(0.0)
    slErr = ctypes.c_short(0)

    _GeoTrans(
        ctypes.byref(slSys1), ctypes.byref(slZone1),
        ctypes.byref(x1), ctypes.byref(y1), ctypes.byref(z1),
        ctypes.byref(slSys2), ctypes.byref(slZone2),
        ctypes.byref(x2), ctypes.byref(y2), ctypes.byref(z2),
        ctypes.byref(slErr)
    )

    if slErr.value != 0:
        raise RuntimeError(f"Transformation failed. Error Code: {slErr.value}")

    return y2.value, x2.value


def create_world_file(image_path, x_min_trans, y_min_trans, x_max_trans, y_max_trans):
    img = Image.open(image_path)
    width, height = img.size
    pixel_size_x = (x_max_trans - x_min_trans) / width
    pixel_size_y = (y_min_trans - y_max_trans) / height
    top_left_easting = x_min_trans
    top_left_northing = y_max_trans

    world_file_path = os.path.splitext(image_path)[0] + ".jgw"
    with open(world_file_path, 'w') as wf:
        wf.write(f"{pixel_size_x}\n")
        wf.write("0.0\n")
        wf.write("0.0\n")
        wf.write(f"{pixel_size_y}\n")
        wf.write(f"{top_left_easting}\n")
        wf.write(f"{top_left_northing}\n")

    print(f"World file created: {world_file_path}")


def process_folder(base_folder, dll_path, href_file):
    for root, _, files in os.walk(base_folder):
        jpg_files = {os.path.splitext(f)[0]: os.path.join(root, f) for f in files if f.endswith(".jpg")}
        sos_files = {os.path.splitext(f)[0]: os.path.join(root, f) for f in files if f.endswith(".sos")}

        matching_files = set(jpg_files.keys()) & set(sos_files.keys())
        for match in matching_files:
            jpg_path = jpg_files[match]
            sos_path = sos_files[match]

            koordsys, coords = parse_sos_file(sos_path)
            if len(coords) != 2:
                print(f"Skipping {sos_path} - Invalid coordinate pairs")
                continue

            (x_min, y_min), (x_max, y_max) = coords
            x_min, x_max = sorted([x_min, x_max])
            y_min, y_max = sorted([y_min, y_max])

            if koordsys == 110:
                try:
                    trans_dll, _InitSkTrans, _GeoTrans = initialize_transformation(dll_path, href_file)
                    slSys1 = ctypes.c_short(4)
                    slZone1 = ctypes.c_short(1)
                    slSys2 = ctypes.c_short(7)
                    slZone2 = ctypes.c_short(32)

                    def transform(x, y):
                        return transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2)

                    easting_min, northing_min = transform(x_min, y_min)
                    easting_max, northing_max = transform(x_max, y_max)
                except RuntimeError as e:
                    print(f"Error transforming {sos_path}: {e}")
                    continue
            else:
                easting_min, northing_min = x_min, y_min
                easting_max, northing_max = x_max, y_max

            create_world_file(jpg_path, easting_min, northing_min, easting_max, northing_max)


if __name__ == "__main__":
    base_folder = "../dataset"
    dll_path = r"skt2_1507-1504_1.dll"
    href_file = r"HREF2018B_NN2000_EUREF89.bin"

    process_folder(base_folder, dll_path, href_file)


World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200b.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200f.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200d.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200h.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200g.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200e.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200i.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201k.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201b.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201e.jgw
World fi

c:\Users\SivertEspelandHusebø\python32\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (171714688 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


World file created: ../dataset\dataset\aalesund\FOKUS\1504250\250.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504250\250c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504251\251.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504252\252.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504256\256.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504261\261a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504261\261.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504262A\262a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263j.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263i.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263e.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263d.jgw
World file